In [ ]:
# Instalando las paqueterías necesarias.
!pip install pandas openpyxl requests beautifulsoup4
!pip install wget
!pip install gdown
!pip install unidecode

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=490e24604e5c81f9769323390fb85a6301eb15657e50f8fd72e8541294699f02
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.5 MB/s eta 0:00:00


Las siguientes librerías nos ayudaran para el desarrollo del proceso de descarga y generación del archivo txt. es necesario que este al principio para su correcto funcionamiento.

Se utiliza librerías especiales como gwdown para descarga de archivos grandes, eliminando la confirmación de permiso de descarga.

Se utiliza librería unicode, para poder quitar los acentos de ciertas palabras y usarlo como nombre de archivos a descargar.


In [ ]:
# Importación de librerías para usarlas en el proyecto de descarga y generación del txt con la descripción de esta
import pandas as pd
import wget
import gdown
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime
from logging import exception
from unidecode import unidecode

La inicialización de ciertas variables ayudaran a preparar la información inicial de cada una de ellas, además de que puedan ser usadas en cualquier parte del código o proceso, sin en el que perjudique el ámbito o espacio donde estas estén siendo seteadas.

In [ ]:
# Inicializando variables que serán usadas más adelante

incidencia_delictiva_link = 'https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva'
incidencia_title = ''
incidencia_description = ''

# Creando variable que contendra el listado de cada source descargado.
sources_list = []

texto_list = ''

In [ ]:
# sección de funciones para descarga de archivos desde fuentes que vienen de enlaces proporcionados por la página del gobierno.

# Función utilizada para la descarga de archivos almacenados en google drive por parte de la institución donde se requiere obtener los datos.
# - document: recibe el id del documento en google drive.
# - description: Texto de la etiqueta de enlace, el cual describe la naturaleza de la información a descargar.
def download_csv_from_http(document, description):

  print(f"link " + description)

  try:

    # Url de descarga a traves de google drive
    csv_url = f'https://drive.google.com/uc?id={document}'

    # Descargando archivos desde la url proporcionada
    response_csv = requests.get(csv_url)

    # Parte donde se hace una limpieza del parámetro description enviado,
    # esto para construir el nombre que tendrá el archivo csv que se descarga a partir de los datos de las fuentes
    # Utilizando expresión regular para reemplazar caracteres especiales y espacios por un solo guion bajo aplicándolo en la descripción del enlace.
    csv_filename = re.sub(r'[^\w\s]+', '', description)
    csv_filename = csv_filename.strip().replace(" ", "")
    # Utilizando unidecode para quitar los acentos y no cause conflictos como nombre de archivo
    csv_filename = unidecode(csv_filename)
    csv_filename = csv_filename + ".csv"
    #------------------------------------------------------------------------------------------------------------------------------------

    # Guardando el archivo descargado, en el repositorio del proyecto
    with open(csv_filename, 'wb') as csv_file:
      csv_file.write(response_csv.content)
  except exception as err:
    print(f'Genero error: ' + description)

# *****************************************************************************************************************************************

# Función utilizada para la descarga de archivos almacenados en google drive por parte de la institución donde se requiere obtener los datos.
# - document: recibe el id del documento en google drive.
# - description: Texto de la etiqueta de enlace, el cual describe la naturaleza de la información a descargar.
# (Para descarga de documentos muy grandes, libreria gdown)
def download_csv_from_http_high(document, description):

  # Url de descarga a traves de google drive
  csv_url = f'https://drive.google.com/uc?id={document}'

  # Parte donde se hace una limpieza del parámetro description enviado,
  # esto para construir el nombre que tendrá el archivo csv que se descarga a partir de los datos de las fuentes
  # Utilizando expresión regular para reemplazar caracteres especiales y espacios por un solo guion bajo aplicándolo en la descripción del enlace.
  csv_filename = re.sub(r'[^\w\s]+', '', description)
  csv_filename = csv_filename.strip()
  csv_filename = csv_filename + ".csv"

  #----------------------------------------------------------------------------------------------------------------------------------------

  # Utilizando wget para descargar archivos grandes sin restricciones
  gdown.download(csv_url, csv_filename, quiet=False)

In [ ]:
# Descargando archivos proporcionados por https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva
print('# Descargando archivos proporcionados por https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva')
# Realizando Web scraping a la página donde están las distintas fuentes de datos
# Esto asegura que los datos puedan ser descargados de forma automática sin necesidad de estar descargado uno por uno directamente desde la página.
url = 'https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
pretty_soup = soup.prettify()

# Obteniendo el texto dentro de la etiqueta <title>
incidencia_source_title = soup.title.text.strip()

# Separando el texto por los pipes "|", esto porque el texto original viene con dicho carácter especial
title_parts = incidencia_source_title.split('|')

# Limpiando los espacios en blanco alrededor de cada parte del titulo encontrado
title_parts = [part.strip() for part in title_parts]

incidencia_title = title_parts[0] + " - " + title_parts[1]

# Buscando todas las etiquetas <h2> y extraer su texto, esto ayudara a poder extraer la descripción de la página, la explicación de la razón de la página.
h2_tags = soup.find_all('h2')

# Se recorren cada una de los parrafos H2 encontrados para luego extraer el texto en donde comience con "En esta pagina...".
for h2_tag in h2_tags:
    text_inside_h2 = h2_tag.text.strip()

    if(text_inside_h2.find('En esta página')!=-1) :
      incidencia_description = text_inside_h2

# Buscando todas las etiquetas <ul>, esto ayudara más adelante a poder realizar las descargas, pues aquí se encontrará las url de cada enlace.
ul_tags = soup.find_all('ul')

for ul_tag in ul_tags:
    # Buscando todas las etiquetas <li> dentro de cada <ul>
    li_tags = ul_tag.find_all('li')

    for li_tag in li_tags:
         # Buscando todas las etiquetas <a> dentro de cada <li> con texto que comienza con "Cifras"
        a_tags = li_tag.find_all('a', href=True, string=lambda x: x and x.startswith('Cifras'))

        for a_tag in a_tags:
            # Obteniendo el valor del atributo href, para extraer su url
            href_value = a_tag['href']

            # Utilizando expresión regular para extraer la parte antes de "/view" de cada url esto para conseguir el id del documento
            match = re.search(r'/file/d/(.*?)/view', href_value)
            # match = re.search(r'(.*?)/view', href_value)

            # En caso de encontrar coicidencia, se procede a extraer el id del documento de descarga.
            if match:
              extracted_part = match.group(1)
              if a_tag.text.find(str(datetime.now().year)) != -1:
                # Condicionante para poder detectar si la descarga es de datos referentes a municipios, esto por conocimiento, son descargas más pesadas y por lo cual se ocupa de librería especial.
                if a_tag.text.find('Municipal') != -1:
                  download_csv_from_http_high(extracted_part, a_tag.text)

                  # Imprimiendo el tag descriptivo y la url para mostrarla en pantalla
                  print(f""+ a_tag.text + ": " + href_value)
                else:
                  download_csv_from_http(extracted_part, a_tag.text)

                #Creando listado que será usado para el documento con la descripción de la descarga de las distintas fuentes, de incidencia delictiva
                sources_list.append(f"- {a_tag.text}: {href_value}")

# Descargando archivos proporcionados por https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva
link Cifras de Incidencia Delictiva Estatal, 2015 - agosto  2023


Downloading...
From: https://drive.google.com/uc?id=13TjyJ9RkR49o0eWTFvhNqazeL4maORYp
To: /content/Cifras de Incidencia Delictiva Municipal 2015  agosto 2023.csv
100%|██████████| 306M/306M [00:02<00:00, 123MB/s]


Cifras de Incidencia Delictiva Municipal, 2015 - agosto 2023. : https://drive.google.com/file/d/13TjyJ9RkR49o0eWTFvhNqazeL4maORYp/view?usp=sharing
link Cifras de Víctimas del Fuero Común, 2015 - agosto 2023
link Cifras de Incidencia Delictiva Federal, 2012 - agosto 2023


In [ ]:
# Leyendo archivos en DataFrames, a partir de los archivos csv generados, provenientes de las fuentes de datos.
df_incidencia = pd.read_csv('Cifras de Incidencia Delictiva Municipal 2015  agosto 2023.csv', encoding="latin1")

print(df_incidencia.shape)

(2075738, 21)


In [ ]:
# Creando el archivo de texto con descripción general de las fuentes de descargar y parte del proceso ETL

# Uniendo elementos de la lista de las fuentes en donde se han descargado los datos (sources_list)
texto_list = "\n".join(sources_list)

date_now = datetime.now()

# Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
description = f"""
Proyecto: contando una historia a través de los datos (análisis de la variabilidad del comportamiento delictivo
en relación con factores temporales y geográficos, y su correlación con el desarrollo social y urbano para la prevención del delito)

Equipo participante:
- Ernesto
- Luis Andrés
- Mario Estrada Ferreira

Descripción de los datos descargados:
- Las fuentes fueron descargadas y donde se puede encontrar información adicional en el sitio web: {incidencia_delictiva_link}

Fuentes:
{texto_list}

Fechas de descarga:
- El proceso de descarga de los datos se realizó el: {date_now.strftime('%Y-%m-%d %H:%M:%S')}
"""
# Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
with open(f'fuentes_descripcion{date_now.strftime("%d%m%y")}.txt', 'w') as description_file:
    description_file.write(description)

print("Archivos de datos descargados y descripción en txt creada.")

Archivos de datos descargados y descripción en txt creada.


# Creación de DataFrames


## Primer DataFrame: Incidencia Delictiva a Nivel Estatal

In [ ]:
# Leemos el archivos csv para obtener el dataframe original
df1 = pd.read_csv('/content/CifrasdeIncidenciaDelictivaEstatal2015agosto 2023.csv', encoding='latin-1')
df1.columns

Index(['Año', 'Clave_Ent', 'Entidad', 'Bien jurídico afectado',
       'Tipo de delito', 'Subtipo de delito', 'Modalidad', 'Enero', 'Febrero',
       'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre',
       'Octubre', 'Noviembre', 'Diciembre'],
      dtype='object')

In [ ]:
mes = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
         'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

df1 = df1.reset_index()

# Obtenemos los datos referentes al estado de Sonora y pasamos de un formato wide a uno long
df1 = df1[df1['Entidad'] == 'Sonora']
df1 = pd.melt(df1, id_vars = list(df1.columns[0:7]), value_vars = mes, var_name = 'Mes', value_name = 'Número de delitos')
df1 = df1.drop('index', axis = 1)

# Checamos que columnas tienen valores faltantes
for column in df1.columns:
  print(f'Columna {column}: ' + str(df1[column].isnull().values.any()))

Columna Año: False
Columna Clave_Ent: False
Columna Entidad: False
Columna Bien jurídico afectado: False
Columna Tipo de delito: False
Columna Subtipo de delito: False
Columna Mes: False
Columna Número de delitos: True


In [ ]:
# Quitamos observaciones en el futuro o no registradas, esto es, observaciones de los meses de Septiembre, Octubre, NOviembre
# y Diciembre del año 2023.
df1 = df1.drop(df1[df1['Mes'].isin(['Septiembre', 'Octubre', 'Noviembre', 'Diciembre']) & (df1['Año'].astype('string') == '2023')].index)
df1.rename(columns = {'Año': 'Ciclo'}, inplace = True)

In [ ]:
df1.head()
df1.shape

(10192, 8)

In [ ]:
# Checamos nuevamente si existen datos faltantes

for column in df1.columns:
  print(f'Columna {column}: ' + str(df1[column].isnull().values.any()))

Columna Ciclo: False
Columna Clave_Ent: False
Columna Entidad: False
Columna Bien jurídico afectado: False
Columna Tipo de delito: False
Columna Subtipo de delito: False
Columna Mes: False
Columna Número de delitos: False


## Segundo DataFrame: Incidencia delictiva a nivel municipal 2015-2023

In [ ]:
# Leemos el archivo csv para obtener el dataframe original

df2 = pd.read_csv('/content/Cifras de Incidencia Delictiva Municipal 2015  agosto 2023.csv', encoding='latin-1')
df2.head()

,Año,Clave_Ent,Entidad,Cve. Municipio,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,...,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,2,...,1,1,0,1,1,0,2.0,1.0,0.0,1.0
1,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,1,...,0,0,0,1,0,1,0.0,0.0,0.0,0.0
2,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0,...,1,1,3,2,0,1,2.0,0.0,0.0,0.0
3,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,1,...,0,1,0,0,0,0,0.0,0.0,0.0,0.0
4,2015,1,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,0,...,0,0,1,0,0,0,0.0,0.0,0.0,0.0


In [ ]:
# Obtenemos un DataFrame para el estado de Sonora
df2s = df2[df2['Entidad'] == 'Sonora']
df2s = df2s.reset_index()
df2s = pd.melt(df2s, id_vars = list(df2.columns[0:9]), value_vars = mes, var_name = 'Mes', value_name = 'Número de delitos')

for column in df2s.columns:
  print(f'Columna {column}: ' + str(df2s[column].isnull().values.any()))

Columna Año: False
Columna Clave_Ent: False
Columna Entidad: False
Columna Cve. Municipio: False
Columna Municipio: False
Columna Bien jurídico afectado: False
Columna Tipo de delito: False
Columna Subtipo de delito: False
Columna Modalidad: False
Columna Mes: False
Columna Número de delitos: True


In [ ]:
# Quitamos observaciones en el futuro o no registradas, esto es, observaciones de los meses de Septiembre, Octubre, NOviembre
# y Diciembre del año 2023.
df2s = df2s.drop(df2s[df2s['Mes'].isin(['Septiembre', 'Octubre', 'Noviembre', 'Diciembre']) & (df2s['Año'].astype('string') == '2023')].index)

In [ ]:
df2s.head()
df2s.shape

(754208, 11)

In [ ]:
# Verificamos si existen otros datos faltantes
for column in df2s.columns:
  print(f'Columna {column}: ' + str(df2s[column].isnull().values.any()))

Columna Año: False
Columna Clave_Ent: False
Columna Entidad: False
Columna Cve. Municipio: False
Columna Municipio: False
Columna Bien jurídico afectado: False
Columna Tipo de delito: False
Columna Subtipo de delito: False
Columna Modalidad: False
Columna Mes: False
Columna Número de delitos: False


## Tercer DataFrame: Incidencia delictiva federal 2012-2023

In [ ]:
# Leemos el archivo csv
df3 = pd.read_csv('/content/CifrasdeIncidenciaDelictivaFederal2012agosto 2023.csv', encoding = 'latin-1')
df3s = df3[df3['ENTIDAD'] == 'SONORA']
df3s.head()

,AÑO,INEGI,ENTIDAD,LEY,CONCEPTO,TIPO,ENERO,FEBRERO,MARZO,ABRIL,MAYO,JUNIO,JULIO,AGOSTO,SEPTIEMBRE,OCTUBRE,NOVIEMBRE,DICIEMBRE
400,2012,26.0,SONORA,CODIGO PENAL FEDERAL,CONTRA LA SALUD,PRODUCCION,10,28,19,14,13,16,17,24,13.0,15.0,12.0,14.0
401,2012,26.0,SONORA,CODIGO PENAL FEDERAL,CONTRA LA SALUD,TRANSPORTE,21,8,10,5,11,11,5,3,7.0,7.0,11.0,7.0
402,2012,26.0,SONORA,CODIGO PENAL FEDERAL,CONTRA LA SALUD,TRAFICO,0,0,1,1,1,0,1,0,0.0,0.0,0.0,0.0
403,2012,26.0,SONORA,CODIGO PENAL FEDERAL,CONTRA LA SALUD,COMERCIO,45,8,9,13,16,11,13,10,5.0,7.0,7.0,9.0
404,2012,26.0,SONORA,CODIGO PENAL FEDERAL,CONTRA LA SALUD,SUMINISTRO,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0


In [ ]:
import numpy as np
Mes = list(np.char.upper(np.array(mes)))

df3s = df3s.reset_index()
df3s = pd.melt(df3s, id_vars = list(df3.columns[0:6]),
               value_vars = Mes, var_name = 'MES', value_name = 'NÚMERO DE DELITOS')

In [ ]:
df3s.head()

,AÑO,INEGI,ENTIDAD,LEY,CONCEPTO,TIPO,MES,NÚMERO DE DELITOS
0,2012,26.0,SONORA,CODIGO PENAL FEDERAL,CONTRA LA SALUD,PRODUCCION,ENERO,10.0
1,2012,26.0,SONORA,CODIGO PENAL FEDERAL,CONTRA LA SALUD,TRANSPORTE,ENERO,21.0
2,2012,26.0,SONORA,CODIGO PENAL FEDERAL,CONTRA LA SALUD,TRAFICO,ENERO,0.0
3,2012,26.0,SONORA,CODIGO PENAL FEDERAL,CONTRA LA SALUD,COMERCIO,ENERO,45.0
4,2012,26.0,SONORA,CODIGO PENAL FEDERAL,CONTRA LA SALUD,SUMINISTRO,ENERO,1.0


In [ ]:
# Quitamos observaciones en el futuro o no registradas, esto es, observaciones de los meses de Septiembre, Octubre, NOviembre
# y Diciembre del año 2023.
df3s = df3s.drop(df3s[df3s['MES'].isin(['SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']) & (df3s['AÑO'].astype('string') == '2023')].index)

In [ ]:
df3s.head()
df3s.shape

(4032, 8)

In [ ]:
# Verificamos si hay datos faltantes
for column in df3s.columns:
  print(f'Columna {column}: ' + str(df3s[column].isnull().values.any()))

Columna AÑO: False
Columna INEGI: False
Columna ENTIDAD: False
Columna LEY: False
Columna CONCEPTO: False
Columna TIPO: False
Columna MES: False
Columna NÚMERO DE DELITOS: False


## Cuarto DataFrame: Victimas del fuero común 2015-2023

In [ ]:
## Leemos el archivo csv y obtenemos los datos de Sonora
df4 = pd.read_csv('/content/CifrasdeVictimasdelFueroComun2015agosto 2023.csv', encoding = 'latin-1')
df4s = df4[df4['Entidad'] == 'Sonora']
df4s.head()

,Año,Clave_Ent,Entidad,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Sexo,Rango de edad,Enero,...,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
5750,2015,26,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,Mujer,Menores de edad (0-17),0,...,0,0,0,1,0,0,1.0,0.0,0.0,0.0
5751,2015,26,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,Mujer,Menores de edad (0-17),0,...,0,0,0,0,1,0,0.0,0.0,1.0,0.0
5752,2015,26,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,Mujer,Menores de edad (0-17),0,...,0,1,0,0,1,0,0.0,0.0,0.0,0.0
5753,2015,26,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,Mujer,Menores de edad (0-17),0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
5754,2015,26,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,Mujer,Menores de edad (0-17),0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [ ]:
for column in df4s.columns:
  print(f'Columna {column}: ' + str(df4s[column].isnull().values.any()))

Columna Año: False
Columna Clave_Ent: False
Columna Entidad: False
Columna Bien jurídico afectado: False
Columna Tipo de delito: False
Columna Subtipo de delito: False
Columna Modalidad: False
Columna Sexo: False
Columna Rango de edad: False
Columna Enero: False
Columna Febrero: False
Columna Marzo: False
Columna Abril: False
Columna Mayo: False
Columna Junio: False
Columna Julio: False
Columna Agosto: False
Columna Septiembre: True
Columna Octubre: True
Columna Noviembre: True
Columna Diciembre: True


In [ ]:
# Pasamos de formato wide a long
df4s = df4s.reset_index()
df4s = pd.melt(df4s, id_vars = list(df4.columns[0:9]),
               value_vars = mes, var_name = 'Mes', value_name = 'Número de delitos')

In [ ]:
# Quitamos observaciones en el futuro o no registradas, esto es, observaciones de los meses de Septiembre, Octubre, NOviembre
# y Diciembre del año 2023.
df4s = df4s.drop(df4s[df4s['Mes'].isin(['Septiembre', 'Octubre', 'Noviembre', 'Diciembre']) & (df4s['Año'].astype('string') == '2023')].index)

In [ ]:
df4s.head()
df4s.shape

(23920, 11)

In [ ]:
# Verificamos si existen otros datos faltantes
for column in df4s.columns:
  print(f'Columna {column}: ' + str(df4s[column].isnull().values.any()))

Columna Año: False
Columna Clave_Ent: False
Columna Entidad: False
Columna Bien jurídico afectado: False
Columna Tipo de delito: False
Columna Subtipo de delito: False
Columna Modalidad: False
Columna Sexo: False
Columna Rango de edad: False
Columna Mes: False
Columna Número de delitos: False


## DataFrame Clima

In [4]:
import pandas as pd
import datetime as dt
import numpy as np

# Primera versión
dfclima = pd.read_csv('../rawdata/DatosClimaMunicipal.csv')

In [5]:
dfclima.rename(columns = {'time':'Dia', 'temperature_2m_max': 'Temp_max',
                          'temperature_2m_min':'Temp_min', 'sunrise':'Salida_sol',
                          'sunset':'Puesta_sol', 'rain_sum':'Lluvia', 
                          'precipitation_hours': 'Tiempo_precipitacion',
                          'temperature_2m_mean': 'Temperatura_promedio',
                          'precipitation_sum':'Precipitacion_total',
                          'municipio': 'Municipio'}, inplace = True)

In [7]:
fecha_final = np.datetime64('2023-09-29')
dfclima['Dia'] = pd.to_datetime(dfclima['Dia'])
dfclima = dfclima.drop(dfclima[dfclima['Dia'] >= fecha_final].index)

In [8]:
dfclima.head()

,Dia,Temp_max,Temp_min,Temperatura_promedio,Salida_sol,Puesta_sol,Precipitacion_total,Lluvia,Tiempo_precipitacion,Municipio
0,2015-01-01,14.6,8.6,11.0,2015-01-01T07:16,2015-01-01T17:32,0.1,0.1,1.0,Aconchi
1,2015-01-02,12.1,3.4,7.5,2015-01-02T07:16,2015-01-02T17:33,0.0,0.0,0.0,Aconchi
2,2015-01-03,16.3,-0.6,7.4,2015-01-03T07:16,2015-01-03T17:33,0.0,0.0,0.0,Aconchi
3,2015-01-04,22.7,3.5,11.5,2015-01-04T07:16,2015-01-04T17:34,0.0,0.0,0.0,Aconchi
4,2015-01-05,24.3,5.8,13.6,2015-01-05T07:17,2015-01-05T17:35,0.0,0.0,0.0,Aconchi


In [9]:
# Verificamos si existen otros datos faltantes
for column in dfclima.columns:
  print(f'Columna {column}: ' + str(dfclima[column].isnull().values.any()))

Columna Dia: False
Columna Temp_max: False
Columna Temp_min: False
Columna Temperatura_promedio: False
Columna Salida_sol: False
Columna Puesta_sol: False
Columna Precipitacion_total: False
Columna Lluvia: False
Columna Tiempo_precipitacion: False
Columna Municipio: False
